In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point


*IMPORT RELEVANT DATA FOR APARTMENT*

In [3]:
kn_data_stavbe_deli_stavb = pd.read_csv("../data/kataster_stavb/KN_SLO_STAVBE_SLO_20240505/KN_SLO_STAVBE_SLO_deli_stavb_20240505.csv", delimiter=',', error_bad_lines=False)
ks_data_stavbe = gpd.read_file("../data/kataster_stavb/KN_SLO_STAVBE_SLO_20240505/KN_SLO_STAVBE_SLO_stavbe_20240505/KN_SLO_STAVBE_SLO_STAVBE_tocka.dbf", delimiter=',', error_bad_lines=False)
ulica = pd.read_csv("../data/kataster_stavb/KN_SLO_NASLOVI_HS_20240505/KN_SLO_NASLOVI_HS_naslovi_hs_20240505.csv", delimiter=',', error_bad_lines=False)

print('finished')

/tmp/ipykernel_2888/1971879740.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  kn_data_stavbe_deli_stavb = pd.read_csv("../data/kataster_stavb/KN_SLO_STAVBE_SLO_20240505/KN_SLO_STAVBE_SLO_deli_stavb_20240505.csv", delimiter=',', error_bad_lines=False)
/tmp/ipykernel_2888/1971879740.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ulica = pd.read_csv("../data/kataster_stavb/KN_SLO_NASLOVI_HS_20240505/KN_SLO_NASLOVI_HS_naslovi_hs_20240505.csv", delimiter=',', error_bad_lines=False)


finished


*Choose only apartments VRSTA_STANOVANJA_ID == 1 AND VRSTA_PROSTORA_ID == 2*

In [4]:
# Load shapefile into a GeoDataFrame
emv_vrednostne_cone_stanovanj = gpd.read_file('../data/emv_vredn_cone_17_VSE/emv_vredn_cone_STA.shp')

# Ensure EID_STAVBA is an integer type right after loading data
kn_data_stavbe_deli_stavb['EID_STAVBA'] = kn_data_stavbe_deli_stavb['EID_STAVBA'].astype(int)
ks_data_stavbe['EID_STAVBA'] = ks_data_stavbe['EID_STAVBA'].astype(int)

# Merge dataframes on EID_STAVBA
apartments = pd.merge(
    kn_data_stavbe_deli_stavb,
    ks_data_stavbe,
    on='EID_STAVBA'
)

# Filter apartments based on specific criteria
apartments = apartments.query('VRSTA_STANOVANJA_ID == 1')

# Merge additional apartment data with 'ulice' data
all_appartments = pd.merge(
    apartments,
    ulica,
    on=['KO_ID', 'EID_STAVBA', 'ST_STAVBE']
)

all_points = np.array(list(all_appartments.geometry.apply(lambda x: (x.x, x.y))))
#adds columns for position_y and position_x
all_appartments[['position_x','position_y']] = all_points
# Convert to GeoDataFrame, ensure CRS match, and perform spatial join
final_all_data = gpd.GeoDataFrame(all_appartments, geometry='geometry')

final_all_data = final_all_data.sjoin(emv_vrednostne_cone_stanovanj, predicate='within')

In [21]:
final_all_data[(final_all_data['KO_ID']==2223) & ((final_all_data['ST_STAVBE']==74)) & (final_all_data['ST_STANOVANJA']== 1)]

,FEATUREID_x,EID_DEL_STAVBE,EID_STAVBA,EID_HISNA_STEVILKA_x,UPRAVNIK_ID,VRSTA_DEJANSKE_RABE_DEL_ST_ID,DVIGALO,LETO_OBNOVE_INSTALACIJ,LETO_OBNOVE_OKEN,POVRSINA,...,KOHEZIJSKA_REGIJA_NAZIV_DJ,DATUM_SYS,position_x,position_y,index_right,IME,MODEL,ID,ST_RAVNI,DAT_VELJ
658956,DELI_STAVB.100300000287001262,100300000287001262,100200000101477798,1.004000e+17,NaN,2,2,2008.0,2008.0,88.1,...,NaN,2024-02-29T17:40:09,390519.739242,124056.563107,484,STA_1040710,STA,1040710,9,2020-04-01


In [24]:
filtered_data = final_all_data[(final_all_data['KO_ID'] == 2223) &
                               (final_all_data['ST_STAVBE'] == 74) &
                               (final_all_data['ST_DELA_STAVBE'] == 1)]

# Viewing the filtered DataFrame
# To display all columns, adjust the display settings (optional, depending on your environment)
pd.set_option('display.max_columns', None)
filtered_data

,FEATUREID_x,EID_DEL_STAVBE,EID_STAVBA,EID_HISNA_STEVILKA_x,UPRAVNIK_ID,VRSTA_DEJANSKE_RABE_DEL_ST_ID,DVIGALO,LETO_OBNOVE_INSTALACIJ,LETO_OBNOVE_OKEN,POVRSINA,STATUS,ST_DELA_STAVBE,EID_STANOVANJE,ST_STANOVANJA,VRSTA_STANOVANJA_ID,UPORABNA_POVRSINA,DATUM_SYS_x,PROSTORNINA_CISTERN_SILOSOV,NACIN_DOLOCITVE_POVRSINE_DELA_STAVBE_ID,UPRAVNIK_STATUS,ETAZNA_LASTNINA,SKUPNI_DEL_ETAZNA_LASTNINA,FEATUREID_y,KO_ID,ST_STAVBE,STEVILO_ET,STEVILO_ST,TIP_STAVBE,ELEKTRIKA,PLIN,VODOVOD,KANALIZACI,LETO_IZGRA,LETO_OBNOV,LETO_OBNO0,VISINA_H1,VISINA_H2,VISINA_H3,POLOZAJNA_,VISINSKA_N,NOSILNA_KO,DATUM_SYS_y,VISINSKI_D,STATUS_VPI,STEVILO_PO,BRUTO_TLOR,EID_OBCINA_x,geometry,FEATUREID,EID_HISNA_STEVILKA_y,OBCINA_SIFRA,OBCINA_NAZIV,OBCINA_NAZIV_DJ,NASELJE_SIFRA,NASELJE_NAZIV,NASELJE_NAZIV_DJ,ULICA_SIFRA,ULICA_NAZIV,ULICA_NAZIV_DJ,POSTNI_OKOLIS_SIFRA,POSTNI_OKOLIS_NAZIV,POSTNI_OKOLIS_NAZIV_DJ,HS_STEVILKA,HS_DODATEK,E,N,EID_OBCINA_y,EID_NASELJE,EID_ULICA,EID_POSTNI_OKOLIS,EID_CETRTNA_SKUPNOST,CETRTNA_SKUPNOST_NAZIV,CETRTNA_SKUPNOST_NAZIV_DJ,EID_DZ_VOLISCE,DZ_VOLISCE_NAZIV,DZ_VOLISCE_NAZIV_DJ,EID_KRAJEVNA_SKUPNOST,KRAJEVNA_SKUPNOST_NAZIV,KRAJEVNA_SKUPNOST_NAZIV_DJ,EID_LOKALNO_VOLISCE,LOKALNO_VOLISCE_NAZIV,LOKALNO_VOLISCE_NAZIV_DJ,EID_LOKALNA_VOLILNA_ENOTA,LOKALNA_VOLILNA_ENOTA_NAZIV,LOKALNA_VOLILNA_ENOTA_NAZIV_DJ,EID_SOLSKI_OKOLIS,SOLSKI_OKOLIS_NAZIV,SOLSKI_OKOLIS_NAZIV_DJ,EID_STATISTICNA_REGIJA,STATISTICNA_REGIJA_NAZIV,STATISTICNA_REGIJA_NAZIV_DJ,EID_UPRAVNA_ENOTA,UPRAVNA_ENOTA_NAZIV,UPRAVNA_ENOTA_NAZIV_DJ,EID_VASKA_SKUPNOST,VASKA_SKUPNOST_NAZIV,VASKA_SKUPNOST_NAZIV_DJ,EID_VOLILNA_ENOTA_DZ,VOLILNA_ENOTA_DZ_NAZIV,VOLILNA_ENOTA_DZ_NAZIV_DJ,EID_VOLILNI_OKRAJ,VOLILNI_OKRAJ_NAZIV,VOLILNI_OKRAJ_NAZIV_DJ,EID_KOHEZIJSKA_REGIJA,KOHEZIJSKA_REGIJA_NAZIV,KOHEZIJSKA_REGIJA_NAZIV_DJ,DATUM_SYS,position_x,position_y,index_right,IME,MODEL,ID,ST_RAVNI,DAT_VELJ
658956,DELI_STAVB.100300000287001262,100300000287001262,100200000101477798,1.004000e+17,NaN,2,2,2008.0,2008.0,88.1,1,1,1.012000e+17,1.0,1.0,88.1,2024-03-03T08:10:04,NaN,1,NaN,1,2,STAVBE.100200000101477798,2223.0,74.0,4.0,13.0,1.0,1.0,2.0,1.0,1.0,1900.0,1950.0,2000.0,233.4,246.0,235.9,1,0,3,2023-03-07,1,1,4.0,2141.3,110200000110269581,POINT (390519.739 124056.563),NASLOVI_HS.100400000152798850,100400000152798850,46,Kobarid,NaN,10,Kobarid,NaN,2.0,Gregorčičeva ulica,NaN,5222,Kobarid,NaN,6,NaN,390515.0,124059.0,110200000110269581,110300000101407900,1.104000e+17,111000000214299534,NaN,NaN,NaN,111400000198935885,"Dom A. Manfrede Kobarid, Gregorčičeva ul. 20",NaN,1.105000e+17,Kobarid,NaN,111600000198944950,"Dom Andreja Manfrede Kobarid, Gregorčičeva ul. 20",NaN,111500000240599795,Volilna enota 1,NaN,111100000198928081,OŠ Simona Gregorčiča Kobarid,NaN,110900000214282483,Goriška,NaN,110800000100524080,Tolmin,NaN,NaN,NaN,NaN,111200000108580625,Volilna enota Postojna,NaN,111300000108583131,Volilni okraj Tolmin,NaN,111700000000000024,Zahodna Slovenija,NaN,2024-02-29T17:40:09,390519.739242,124056.563107,484,STA_1040710,STA,1040710,9,2020-04-01


In [20]:
print(list(final_all_data.columns))


['FEATUREID_x', 'EID_DEL_STAVBE', 'EID_STAVBA', 'EID_HISNA_STEVILKA_x', 'UPRAVNIK_ID', 'VRSTA_DEJANSKE_RABE_DEL_ST_ID', 'DVIGALO', 'LETO_OBNOVE_INSTALACIJ', 'LETO_OBNOVE_OKEN', 'POVRSINA', 'STATUS', 'ST_DELA_STAVBE', 'EID_STANOVANJE', 'ST_STANOVANJA', 'VRSTA_STANOVANJA_ID', 'UPORABNA_POVRSINA', 'DATUM_SYS_x', 'PROSTORNINA_CISTERN_SILOSOV', 'NACIN_DOLOCITVE_POVRSINE_DELA_STAVBE_ID', 'UPRAVNIK_STATUS', 'ETAZNA_LASTNINA', 'SKUPNI_DEL_ETAZNA_LASTNINA', 'FEATUREID_y', 'KO_ID', 'ST_STAVBE', 'STEVILO_ET', 'STEVILO_ST', 'TIP_STAVBE', 'ELEKTRIKA', 'PLIN', 'VODOVOD', 'KANALIZACI', 'LETO_IZGRA', 'LETO_OBNOV', 'LETO_OBNO0', 'VISINA_H1', 'VISINA_H2', 'VISINA_H3', 'POLOZAJNA_', 'VISINSKA_N', 'NOSILNA_KO', 'DATUM_SYS_y', 'VISINSKI_D', 'STATUS_VPI', 'STEVILO_PO', 'BRUTO_TLOR', 'EID_OBCINA_x', 'geometry', 'FEATUREID', 'EID_HISNA_STEVILKA_y', 'OBCINA_SIFRA', 'OBCINA_NAZIV', 'OBCINA_NAZIV_DJ', 'NASELJE_SIFRA', 'NASELJE_NAZIV', 'NASELJE_NAZIV_DJ', 'ULICA_SIFRA', 'ULICA_NAZIV', 'ULICA_NAZIV_DJ', 'POSTNI_OK

In [23]:
final_all_data[final_all_data['VRSTA_DEJANSKE_RABE_DEL_ST_ID'] == 2]

,FEATUREID_x,EID_DEL_STAVBE,EID_STAVBA,EID_HISNA_STEVILKA_x,UPRAVNIK_ID,VRSTA_DEJANSKE_RABE_DEL_ST_ID,DVIGALO,LETO_OBNOVE_INSTALACIJ,LETO_OBNOVE_OKEN,POVRSINA,STATUS,ST_DELA_STAVBE,EID_STANOVANJE,ST_STANOVANJA,VRSTA_STANOVANJA_ID,UPORABNA_POVRSINA,DATUM_SYS_x,PROSTORNINA_CISTERN_SILOSOV,NACIN_DOLOCITVE_POVRSINE_DELA_STAVBE_ID,UPRAVNIK_STATUS,ETAZNA_LASTNINA,SKUPNI_DEL_ETAZNA_LASTNINA,FEATUREID_y,KO_ID,ST_STAVBE,STEVILO_ET,STEVILO_ST,TIP_STAVBE,ELEKTRIKA,PLIN,VODOVOD,KANALIZACI,LETO_IZGRA,LETO_OBNOV,LETO_OBNO0,VISINA_H1,VISINA_H2,VISINA_H3,POLOZAJNA_,VISINSKA_N,NOSILNA_KO,DATUM_SYS_y,VISINSKI_D,STATUS_VPI,STEVILO_PO,BRUTO_TLOR,EID_OBCINA_x,geometry,FEATUREID,EID_HISNA_STEVILKA_y,OBCINA_SIFRA,OBCINA_NAZIV,OBCINA_NAZIV_DJ,NASELJE_SIFRA,NASELJE_NAZIV,NASELJE_NAZIV_DJ,ULICA_SIFRA,ULICA_NAZIV,ULICA_NAZIV_DJ,POSTNI_OKOLIS_SIFRA,POSTNI_OKOLIS_NAZIV,POSTNI_OKOLIS_NAZIV_DJ,HS_STEVILKA,HS_DODATEK,E,N,EID_OBCINA_y,EID_NASELJE,EID_ULICA,EID_POSTNI_OKOLIS,EID_CETRTNA_SKUPNOST,CETRTNA_SKUPNOST_NAZIV,CETRTNA_SKUPNOST_NAZIV_DJ,EID_DZ_VOLISCE,DZ_VOLISCE_NAZIV,DZ_VOLISCE_NAZIV_DJ,EID_KRAJEVNA_SKUPNOST,KRAJEVNA_SKUPNOST_NAZIV,KRAJEVNA_SKUPNOST_NAZIV_DJ,EID_LOKALNO_VOLISCE,LOKALNO_VOLISCE_NAZIV,LOKALNO_VOLISCE_NAZIV_DJ,EID_LOKALNA_VOLILNA_ENOTA,LOKALNA_VOLILNA_ENOTA_NAZIV,LOKALNA_VOLILNA_ENOTA_NAZIV_DJ,EID_SOLSKI_OKOLIS,SOLSKI_OKOLIS_NAZIV,SOLSKI_OKOLIS_NAZIV_DJ,EID_STATISTICNA_REGIJA,STATISTICNA_REGIJA_NAZIV,STATISTICNA_REGIJA_NAZIV_DJ,EID_UPRAVNA_ENOTA,UPRAVNA_ENOTA_NAZIV,UPRAVNA_ENOTA_NAZIV_DJ,EID_VASKA_SKUPNOST,VASKA_SKUPNOST_NAZIV,VASKA_SKUPNOST_NAZIV_DJ,EID_VOLILNA_ENOTA_DZ,VOLILNA_ENOTA_DZ_NAZIV,VOLILNA_ENOTA_DZ_NAZIV_DJ,EID_VOLILNI_OKRAJ,VOLILNI_OKRAJ_NAZIV,VOLILNI_OKRAJ_NAZIV_DJ,EID_KOHEZIJSKA_REGIJA,KOHEZIJSKA_REGIJA_NAZIV,KOHEZIJSKA_REGIJA_NAZIV_DJ,DATUM_SYS,position_x,position_y,index_right,IME,MODEL,ID,ST_RAVNI,DAT_VELJ
62,DELI_STAVB.100300000293167693,100300000293167693,100200000250872278,1.004000e+17,293.0,2,2,NaN,1993.0,45.9,1,1,1.012000e+17,3.0,1.0,39.9,2022-05-30T04:36:33,NaN,1,2.0,1,2,STAVBE.100200000250872278,2541.0,365.0,2.0,4.0,1.0,1.0,0.0,1.0,1.0,1600.0,1998.0,1997.0,443.9,453.5,443.9,1,0,3,2022-05-30,1,1,1.0,NaN,110200000110277170,POINT (442322.040 45440.340),NASLOVI_HS.100400000118183585,100400000118183585,38,Ilirska Bistrica,NaN,7,Dolnji Zemon,NaN,NaN,NaN,NaN,6250,Ilirska Bistrica,NaN,58,NaN,442322.0,45439.0,110200000110277170,110300000100970023,NaN,111000000214296787,NaN,NaN,NaN,111400000169238269,Grad Dol. Zemon - prostori KS,NaN,1.105000e+17,Dolnji Zemon,NaN,111600000169255600,Grad Dolnji Zemon -prostori KS,NaN,111500000240597310,Volilna enota 1,NaN,111100000169180084,OŠ A. Žnideršiča Il. Bistrica,NaN,110900000214282301,Primorsko-notranjska,NaN,110800000100519759,Ilirska Bistrica,NaN,NaN,NaN,NaN,111200000108580625,Volilna enota Postojna,NaN,111300000108583727,Volilni okraj Ilirska Bistrica,NaN,111700000000000016,Vzhodna Slovenija,NaN,2024-02-29T17:40:09,442322.04,45440.34,125,STA_1040520,STA,1040520,3,2020-04-01
63,DELI_STAVB.100300000293167701,100300000293167701,100200000250872278,1.004000e+17,293.0,2,2,NaN,1993.0,66.9,1,2,1.012000e+17,1.0,1.0,64.9,2022-05-30T04:36:33,NaN,1,2.0,1,2,STAVBE.100200000250872278,2541.0,365.0,2.0,4.0,1.0,1.0,0.0,1.0,1.0,1600.0,1998.0,1997.0,443.9,453.5,443.9,1,0,3,2022-05-30,1,1,1.0,NaN,110200000110277170,POINT (442322.040 45440.340),NASLOVI_HS.100400000118183585,100400000118183585,38,Ilirska Bistrica,NaN,7,Dolnji Zemon,NaN,NaN,NaN,NaN,6250,Ilirska Bistrica,NaN,58,NaN,442322.0,45439.0,110200000110277170,110300000100970023,NaN,111000000214296787,NaN,NaN,NaN,111400000169238269,Grad Dol. Zemon - prostori KS,NaN,1.105000e+17,Dolnji Zemon,NaN,111600000169255600,Grad Dolnji Zemon -prostori KS,NaN,111500000240597310,Volilna enota 1,NaN,111100000169180084,OŠ A. Žnideršiča Il. Bistrica,NaN,110900000214282301,Primorsko-notranjska,NaN,110800000100519759,Ilirska Bistrica,NaN,NaN,NaN,NaN,111200000108580625,Volilna enota Postojna,NaN,111300000108583727,Volilni okraj Ilirska Bistrica,NaN,11170000000000001